# Neural Network Model


In [31]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# Import and read the csv.
# b2 view
df = pd.read_csv("../ML_Data_&_Preprocessing/b1_df_nonEncoded.csv", index_col=[0])
df.head()


# # b2 view
# df = pd.read_csv("../ML_Data_&_Preprocessing/b2_df_nonEncoded.csv", index_col=[0])
# df.head()

# # b3 view
# df = pd.read_csv("../ML_Data_&_Preprocessing/b3_df_nonEncoded.csv", index_col=[0])
# df.head()


,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
0,CA,40.27,1397,1,17,1320,NaN
1,TX,16.67,877,0,7,816,NaN
2,UT,33.96,1455,0,8,1361,NaN
3,TX,32.97,1554,1,4,1446,NaN
4,VA,32.00,1326,0,4,1242,NaN


In [32]:
df['leads'].value_counts()

1    36788
0    20868
Name: leads, dtype: int64

In [33]:
df2 = df.loc[df['state']== 'CA']
df2

,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
0,CA,40.27,1397,1,17,1320,NaN
5,CA,50.27,2969,1,24,2766,NaN
7,CA,63.96,3025,1,10,2818,NaN
9,CA,88.15,4732,1,38,4497,NaN
10,CA,54.42,1981,1,19,1792,NaN
...,...,...,...,...,...,...,...
57648,CA,25.04,751,0,7,690,1.0
57650,CA,33.18,1200,1,8,1032,1.0
57652,CA,35.91,1852,1,17,1608,1.0
57654,CA,35.24,2019,1,14,1825,1.0


In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16032 entries, 0 to 57655
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         16032 non-null  object 
 1   spend         16032 non-null  float64
 2   Impressions   16032 non-null  int64  
 3   leads         16032 non-null  int64  
 4   link_clicks   16032 non-null  int64  
 5   reach         16032 non-null  int64  
 6   Agency Tiers  1789 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 1002.0+ KB


In [35]:
df3 = df2.drop(df2.columns[[0,6]], axis=1)

In [36]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16032 entries, 0 to 57655
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   spend        16032 non-null  float64
 1   Impressions  16032 non-null  int64  
 2   leads        16032 non-null  int64  
 3   link_clicks  16032 non-null  int64  
 4   reach        16032 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 751.5 KB


# Split into train and test features

In [37]:
# Split our preprocessed data into our features and target arrays
y = df3['leads'].values
X = df3.drop(['leads'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\dia78039\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, Evaluate our Model 1 - NN

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# I have X amount of columns
# layer1 = input layer, typically equals number of input variables in data
layer1 = 30
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 15
# layer 3 = hidden layer
layer3= 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="hard_sigmoid")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="elu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='elu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="hard_sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 30)                150       
                                                                 
 dense_5 (Dense)             (None, 15)                465       
                                                                 
 dense_6 (Dense)             (None, 2)                 32        
                                                                 
 dense_7 (Dense)             (None, 1)                 3         
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [41]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
375/376 [============================>.] - ETA: 0s - loss: 0.5284 - accuracy: 0.7423
Epoch 1: saving model to checkpoints\weights.01.hdf5
376/376 [==============================] - 1s 704us/step - loss: 0.5282 - accuracy: 0.7424
Epoch 2/100
370/376 [============================>.] - ETA: 0s - loss: 0.4656 - accuracy: 0.7874
Epoch 2: saving model to checkpoints\weights.02.hdf5
376/376 [==============================] - 0s 715us/step - loss: 0.4651 - accuracy: 0.7881
Epoch 3/100
373/376 [============================>.] - ETA: 0s - loss: 0.4580 - accuracy: 0.7900
Epoch 3: saving model to checkpoints\weights.03.hdf5
376/376 [==============================] - 0s 712us/step - loss: 0.4584 - accuracy: 0.7898
Epoch 4/100
372/376 [============================>.] - ETA: 0s - loss: 0.4700 - accuracy: 0.7760
Epoch 4: saving model to checkpoints\weights.04.hdf5
376/376 [==============================] - 0s 712us/step - loss: 0.4699 - accuracy: 0.7760
Epoch 5/100
357/376 [===============

Epoch 35/100
362/376 [===========================>..] - ETA: 0s - loss: 0.4331 - accuracy: 0.7945
Epoch 35: saving model to checkpoints\weights.35.hdf5
376/376 [==============================] - 0s 736us/step - loss: 0.4334 - accuracy: 0.7952
Epoch 36/100
364/376 [============================>.] - ETA: 0s - loss: 0.4340 - accuracy: 0.7955
Epoch 36: saving model to checkpoints\weights.36.hdf5
376/376 [==============================] - 0s 718us/step - loss: 0.4335 - accuracy: 0.7957
Epoch 37/100
369/376 [============================>.] - ETA: 0s - loss: 0.4327 - accuracy: 0.7957
Epoch 37: saving model to checkpoints\weights.37.hdf5
376/376 [==============================] - 0s 715us/step - loss: 0.4331 - accuracy: 0.7952
Epoch 38/100
370/376 [============================>.] - ETA: 0s - loss: 0.4334 - accuracy: 0.7949
Epoch 38: saving model to checkpoints\weights.38.hdf5
376/376 [==============================] - 0s 718us/step - loss: 0.4331 - accuracy: 0.7949
Epoch 39/100
375/376 [======

Epoch 69/100
364/376 [============================>.] - ETA: 0s - loss: 0.4577 - accuracy: 0.7884
Epoch 69: saving model to checkpoints\weights.69.hdf5
376/376 [==============================] - 0s 723us/step - loss: 0.4568 - accuracy: 0.7887
Epoch 70/100
304/376 [=======================>......] - ETA: 0s - loss: 0.4425 - accuracy: 0.7896
Epoch 70: saving model to checkpoints\weights.70.hdf5
376/376 [==============================] - 0s 695us/step - loss: 0.4445 - accuracy: 0.7882
Epoch 71/100
374/376 [============================>.] - ETA: 0s - loss: 0.4388 - accuracy: 0.7923
Epoch 71: saving model to checkpoints\weights.71.hdf5
376/376 [==============================] - 0s 705us/step - loss: 0.4384 - accuracy: 0.7925
Epoch 72/100
368/376 [============================>.] - ETA: 0s - loss: 0.4365 - accuracy: 0.7942
Epoch 72: saving model to checkpoints\weights.72.hdf5
376/376 [==============================] - 0s 720us/step - loss: 0.4358 - accuracy: 0.7945
Epoch 73/100
370/376 [======

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# when brand 1, all states, accuracy = 36%
# when brand 2, all states, accuracy = 36%
# when brand 3, all states, accuracy = 36%

# B2, CA = 80%
# B1, CA = ? 
# B3, CA = ?


126/126 - 0s - loss: 0.4198 - accuracy: 0.8056 - 156ms/epoch - 1ms/step
Loss: 0.4198281466960907, Accuracy: 0.8056387305259705


In [ ]:
# # Export the model to HDF5 file
# nn.save("AlphabetSoupCharity_optimization.h5")